# Inventory Optimisation Algorithm

Inventory optimisation: to find the right quantity of stock to keep to meet demand while also minimizing costs 

Inventory Optimization Algorithm:
- Demand Forecast: This will be used to estimate future demand for products over a specific time period.
- Reorder Point (ROP): The level of inventory that triggers a reorder to avoid stockouts.
- Economic Order Quantity (EOQ): The optimal quantity of units to order that minimizes total inventory costs.
- Safety Stock: Extra inventory company holds to mitigate the risk of stockout (running out of stock)

For each product, 
1) calculate the EOQ ( quantity to order to minimize cost), 
2) safety stock, ( extra inventory incase out of stock) 
3) and ROP. (level that triggers a reorder)
4) If the current inventory is below the ROP, place an order equal to the EOQ.

Hypothesis: 
1) Products with lower demand variability requires less safety stock compared to products with high demand variability
2) Products with higher weekly demand will require a higher reorder point.
3) Higher demand products will have larger Economic Order Quantities (EOQ), suggesting that these products should be ordered in larger quantities, but less frequently, compared to lower-demand products.

In [2]:
import math
import pandas as pd
import requests
from io import StringIO
import certifi

In [3]:
# Define the Google Drive link and convert it to a direct download link
csv_url = 'https://drive.google.com/file/d/1GB-if5xcQM64dV4O6_H-HjcgOK21Z9j2/view?usp=sharing'
file_id = csv_url.split('/')[-2]
dwn_url = f'https://drive.google.com/uc?id={file_id}'

# Get the CSV data with SSL verification
response = requests.get(dwn_url, verify=certifi.where()).text

# Load the CSV content directly into a pandas DataFrame without saving it
csv_raw = StringIO(response)
data = pd.read_csv(csv_raw)
data['Invoice Date'] = pd.to_datetime(data['Invoice Date'])


In [4]:

# EOQ formula
def calculate_eoq(demand, order_cost, holding_cost):
    if demand > 0:  # Ensure demand is positive
        return math.sqrt((2 * demand * order_cost) / holding_cost)
    return 0  # If demand is zero or negative, EOQ is zero

# Safety Stock formula
def calculate_safety_stock(z, demand_std, lead_time):
    return z * demand_std * math.sqrt(lead_time)

# ROP formula
def calculate_rop(avg_daily_demand, lead_time, safety_stock):
    return (avg_daily_demand * lead_time) + safety_stock




In [5]:
# Convert 'Invoice Date' to datetime format 
data['Invoice Date'] = pd.to_datetime(data['Invoice Date'])

# Create a 'Week' column based on the weekly period of 'Invoice Date'
data['Week'] = data['Invoice Date'].dt.to_period('W')

# Group by 'StockCode' and 'Week' to get weekly demand metrics
weekly_data = data.groupby(['StockCode', 'Week']).agg({'Quantity': 'sum'}).reset_index()

# Group again by 'StockCode' to calculate total demand, average weekly demand, and demand standard deviation
product_demand = weekly_data.groupby('StockCode').agg({
    'Quantity': ['sum', 'mean', 'std']
}).reset_index()

# Rename columns for easier reference
product_demand.columns = ['StockCode', 'TotalDemand', 'AvgWeeklyDemand', 'DemandStdDev']
product_demand

,StockCode,TotalDemand,AvgWeeklyDemand,DemandStdDev
0,10002,1040,54.736842,74.081668
1,10080,495,30.937500,43.308919
2,10120,192,9.600000,10.869562
3,10123C,5,1.666667,1.154701
4,10124A,16,3.200000,1.303840
...,...,...,...,...
3945,gift_0001_20,20,2.500000,3.116775
3946,gift_0001_30,7,1.400000,0.547723
3947,gift_0001_40,3,1.000000,0.000000
3948,gift_0001_50,4,1.000000,0.000000


In [6]:
import numpy as np

# Define constants for EOQ, Safety Stock, and ROP calculations

# Order cost: fixed expense of placing an order, covering things like administrative processing, shipping, and handling
# assume for it to be 50 since in many scenarios, order costs are in the range of tens to hundreds of dollars.
order_cost = 50  

# Holding cost:  cost of storing one unit of inventory for a certain period (usually per year) we assume it costs $2
# helps us keep just the right amount of stock: prevents us from storing too much (overstocking), which would increase storage costs. At the same time, it's not so high that it forces us to keep too little stock.
holding_cost = 2  

# service level: aim to have enough stock to meet demand 95% of the time without stockouts
# 95% service level is commonly used in retail and inventory management, balancing customer satisfaction with inventory costs.
service_level_factor = 1.65  

# Lead time: represents the time between placing an order and receiving the stock
# is often between 1-3 weeks in many retail and manufacturing settings.
lead_time_weeks = 2 

# Apply calculations for each row
product_demand['EOQ'] = product_demand['TotalDemand'].apply(lambda x: calculate_eoq(x, order_cost, holding_cost))
product_demand['SafetyStock'] = product_demand['DemandStdDev'].apply(lambda x: calculate_safety_stock(service_level_factor, x, lead_time_weeks))
product_demand['ROP'] = product_demand.apply(lambda row: calculate_rop(row['AvgWeeklyDemand'], lead_time_weeks, row['SafetyStock']), axis=1)

product_demand = product_demand.sort_values(by='TotalDemand', ascending=False).reset_index(drop=True)
product_demand['SafetyStock'] = product_demand['SafetyStock'].fillna(0)
product_demand['ROP'] = product_demand['ROP'].fillna(0)
product_demand = product_demand[product_demand['DemandStdDev'] > 0].reset_index(drop=True)
product_demand = product_demand[product_demand['AvgWeeklyDemand'] > 0].reset_index(drop=True)

product_demand


,StockCode,TotalDemand,AvgWeeklyDemand,DemandStdDev,EOQ,SafetyStock,ROP
0,22197,56427,1064.660377,1086.856546,1679.687471,2536.127992,4665.448747
1,84077,53751,1033.673077,979.023698,1639.374881,2284.505176,4351.851329
2,85099B,47260,891.698113,463.652165,1537.205256,1081.910247,2865.306474
3,85123A,39067,737.113208,703.930822,1397.622982,1642.589051,3116.815466
4,84879,36282,684.566038,548.007422,1346.885296,1278.749223,2647.881298
...,...,...,...,...,...,...,...
3657,90176A,2,0.666667,1.527525,10.000000,3.564407,4.897741
3658,35400,2,0.222222,4.763869,10.000000,11.116261,11.560705
3659,23630,1,0.500000,0.707107,7.071068,1.650000,2.650000
3660,22769,1,0.200000,4.868265,7.071068,11.359864,11.759864


### Conclusion:
1) Products with high demand varaibility require more safety stock than those of low demand. 
For example, StockCode 10002 has the highest DemandStdDev (29.69) and highest Safety Stock (129.20) while StockCode 21761 which has the lowest DemandStdDev (0.3333) and the lowest safety stock (0.777817)

2) In general, products with higher average weekly demand require a higher ROP than products with lower average weekly demand. However, there are a few outliers. For example, although stockCode 35400 has a higher AvgWeeklyDemand than stockcode 35400, 35400 gas a higher ROP. This could be due to 35400 having a higher DemandStdDeviation, increasing the safety stock which in turn raises the ROP. 

3)  EOQ (Economic Order Quantity) is directly related to total demand rather than average weekly demand or demand standard deviation. This is because EOQ is calculated based on the annual or total demand. Higher demand products have higher EOQs, indicating that they should be ordered in larger quantities but less frequently to minimize overall costs.
StockCode 22197 that has the highest demand has an EOQ of 1679.687471, while StockCode 22769 that has the lowest demand has an EOQ of 26.46. 

Overall, products with higher demand and variability should be monitored closely, and larger, more frequent orders should be placed, whereas low-demand, low-variability products can be ordered less frequently in smaller quantities.